# LangChain tool calling using Granite-3.0-8B-Instruct in Python with watsonx.ai

**Author**: Anna Gutowska 

In this tutorial, we will use pre-built LangChain tools for an agentic ReAct agent to showcase its ability to differentiate appropriate use cases for each tool. We will primarily be using the open-source [LangChain Python package](https://python.langchain.com/v0.2/docs/introduction/).

# What is tool calling? 
Tool calling, otherwise known as function calling, is the interface that allows AI agents to work on specific tasks that require up-to-date information, otherwise unavailable to the trained large language models (LLMs). LLMs like IBM® [Granite™ models](https://www.ibm.com/products/watsonx-ai/foundation-models) or OpenAI’s GPT (Generative Pre-trained Transformer) models only have access to information used in training. There are many default tools accessible through LangChain including a tool for working with SQL database queries, accessing information on Wikipedia, and much more. We encourage you to read the [LangChain documentaion](https://python.langchain.com/docs/integrations/tools/) for a comprehensive list of pre-built tools. 

Custom tools can be defined using various methods. They can be created using the @tool decorator or LangChain Runnables, as we will cover in this tutorial. Async tools can able be created by using the StructuredTool or the BaseTool classes. For the distinctions between each approach, we encourage you to reference the official [LangChain documentation](https://python.langchain.com/docs/how_to/custom_tools/). Refer to our [function calling tutorial](https://www.ibm.com/think/tutorials/granite-function-calling) for examples of custom tools.

We encourage you to check out our [AI Agents explainer](https://www.ibm.com/think/topics/ai-agents) for an in-depth overview of the various AI agent types and how they differ from traditional LLM chatbots.

# Prerequisites
You need an [IBM Cloud account](https://cloud.ibm.com/registration) project.

# Steps
Check out this [YouTube video](https://www.youtube.com/watch?v=3sav6vUG_XQ) that walks you through the following set up instructions in Steps 1 and 2.

## Step 1. Set up your environment
While you can choose from several tools, this tutorial walks you through how to set up an IBM account to use a Jupyter Notebook. 

1. Log in to [watsonx.ai](https://dataplatform.cloud.ibm.com/registration/stepone?context=wx&apps=all) using your IBM Cloud account.

2. Create a [watsonx.ai project](https://www.ibm.com/docs/en/watsonx/saas?topic=projects-creating-project).

	You can get your project ID from within your project. Click the **Manage** tab. Then, copy the project ID from the **Details** section of the **General** page. You need this ID for this tutorial.

3. Create a [Jupyter Notebook](https://www.ibm.com/docs/en/watsonx/saas?topic=editor-creating-managing-notebooks).

	This step will open a Notebook environment where you can copy the code from this tutorial.  Alternatively, you can download this notebook to your local system and upload it to your watsonx.ai project as an asset. To view more Granite tutorials, check out the [IBM Granite Community](https://github.com/ibm-granite-community). This tutorial is also available on [Github](https://github.com/IBM/ibmdotcom-tutorials/tree/main/generative-ai/langchain-tools.ipynb).

## Step 2. Set up a Watson Machine Learning (WML) service instance and API key.

1. Create a [Watson Machine Learning](https://cloud.ibm.com/catalog/services/watson-machine-learning) service instance (select your appropriate region and choose the Lite plan, which is a free instance).

2. Generate an [API Key in WML](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-authentication.html). 

3. Associate the WML service to the project that you created in [watsonx.ai](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/assoc-services.html). 


## Step 3. Install and import relevant libraries and set up your credentials

We'll need a few libraries and modules for this tutorial. Make sure to import the ones below, and if they're not installed, you can resolve this with a quick pip install. LangChain and LangGraph will be the frameworks and developer toolkits used.

In [ ]:
#installations
!pip install -q langchain \
    "langchain_community<0.3.0" \
    langgraph \
    youtube_search \
    pyowm \
    ionic-langchain \
    python-dotenv
    
!pip install -qU langchain-ibm

In [ ]:
#imports
import os

from langchain_ibm import ChatWatsonx
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import Tool
from langchain_core.messages import HumanMessage
from langchain_community.tools import YouTubeSearchTool
from langchain_community.utilities import OpenWeatherMapAPIWrapper
from ionic_langchain.tool import IonicTool
from dotenv import load_dotenv

load_dotenv(os.getcwd()+"/.env", override=True)

True

To set our credentials, we will need the `WATSONX_APIKEY` and `WATSONX_PROJECT_ID` you generated in step 1. You can either store them in a `.env` file in your directory or replace the placeholder text. We will also set the URL serving as the API endpoint.

In [ ]:
WATSONX_APIKEY = os.getenv('WATSONX_APIKEY', "<WATSONX_APIKEY_HERE>")

WATSONX_PROJECT_ID = os.getenv('WATSONX_PROJECT_ID', "<WATSONX_PROJECT_ID_HERE>")

URL = "https://us-south.ml.cloud.ibm.com"

The weather tool used in this tutorial with require an OpenWeather API key. To generate one, [create an OpenWeather account](https://home.openweathermap.org/users/sign_up). Upon creating an account, select the "API Keys" tab to display your free key.

In [ ]:
OPENWEATHERMAP_API_KEY = os.getenv('OPENWEATHERMAP_API_KEY', "<YOUR_OPEN_WEATHERMAP_API_KEY_HERE>")

## Step 4. Initialize the LLM

For this tutorial, we will be using the ChatWatsonx wrapper to set our chat model. This wrapper simplifies the integration of tool calling and chaining. We will be using the `granite-3-8b-instruct` model. We encourage you to use the API references in the [ChatWatsonx documentation](https://python.langchain.com/docs/integrations/chat/ibm_watsonx/#tool-calling) for further information. 

To initialize the LLM, we need to set the model parameters. It is important to configure the model's `temperature` here in order to limit agent's hallucinations. 

In [ ]:
llm = ChatWatsonx(
    model_id="ibm/granite-3-8b-instruct",
    url = URL,
	apikey = WATSONX_APIKEY,
	project_id = WATSONX_PROJECT_ID,
    params = {
        "decoding_method": "greedy",
        "temperature": 0, 
        "min_new_tokens": 5, 
        "max_new_tokens": 2000
    }
)

## Step 5. Establish the built-in tools

We will use the `Tool` class to make our tools callable. A clear and simple description of the tool is also important. Additionally, the `return_direct` boolean attribute determines whether the tool response should be returned directly to the user. Lastly, the optional `args_schema` attribute of `pydantic.BaseModel` type is used to provide additional information or validation to the model. 

Let's imagine you are planning your next vacation to Greece and are eager to learn more about it and prepare for the trip. First, let's set up the built-in LangChain weather tool. The tool uses the `OpenWeatherMapAPIWrapper` that uses the `OPENWEATHERMAP_API_KEY` we generated earlier. 

In [ ]:
weather = OpenWeatherMapAPIWrapper(openweathermap_api_key=OPENWEATHERMAP_API_KEY)

weather_search = Tool(
    name="weather_search",
    description="Get weather for a city and country code, e.g. Athens, GR",
    func=weather.run,
)

Next, let's set up the pre-built YouTube tool using the YouTube Search package available through LangChain. This will be helpful for finding videos about your travel destination. 

In [ ]:
youtube = YouTubeSearchTool()

youtube_search = Tool(
    name="youtube_search",
    description="Search YouTube for video links.",
    func=youtube.run,
)

Lastly, let's set up an [online shopping tool](https://python.langchain.com/docs/integrations/tools/ionic_shopping/) using [Ionic](https://www.ioniccommerce.com/). This tool returns items relevant to the user query that are on sale through the ecommerce marketplace.

In [ ]:
ionic_search = IonicTool().tool()

Let's set our list of the multiple tools we provide to the LLM. We can also print the tool list to see how they loaded. For an extended list of available LangChain tools, refer to the LangChain [documentation](https://python.langchain.com/docs/integrations/tools/).

In [ ]:
tools = [weather_search, youtube_search, ionic_search]
tools

[Tool(name='weather_search', description='Get weather for a city and country code, e.g. Athens, GR', func=<bound method OpenWeatherMapAPIWrapper.run of OpenWeatherMapAPIWrapper(owm=<pyowm.owm.OWM - API key=************************611450cc, subscription type=free, PyOWM version=(3, 3, 0)>, openweathermap_api_key=None)>),
 Tool(name='youtube_search', description='Search YouTube for video links.', func=<bound method BaseTool.run of YouTubeSearchTool()>),
 Tool(name='ionic_commerce_shopping_tool', description='\nIonic is an e-commerce shopping tool. Assistant uses the Ionic Commerce Shopping Tool to find, discover, and compare products from thousands of online retailers. Assistant should use the tool when the user is looking for a product recommendation or trying to find a specific product. \n\nThe user may specify the number of results, minimum price, and maximum price for which they want to see results.\nIonic Tool input is a comma-separated string of values:\n  - query string (required,

## Step 6. Tool calling

Tool calling typically refers to an LLM returning the name of the tool to call and its arguments. This tool call can then be executed or the extracted information can be used for unrelated purposes. For more examples of this, refer to our [function calling](https://www.ibm.com/think/tutorials/granite-function-calling) tutorial.

Actually running the tool and retrieving its output is not always implied. In this tutorial, we will explore both approaches. 

### Return the relevant tool and arguments

To accomplish traditional tool calling, we can simply provide a user query and use the prebuilt `bind_tools` method to pass the list of tools to the LLM upon each iteration.

In [ ]:
llm_with_tools = llm.bind_tools(tools)
response = llm_with_tools.invoke([
		("human", "What are some youtube videos about greece"),
	])
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-446bafd1e13f48ec9e19b6a8173df71b', 'type': 'function', 'function': {'name': 'youtube_search', 'arguments': '{"query": "greece"}'}}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 578, 'total_tokens': 599}, 'model_name': 'ibm/granite-3-8b-instruct', 'system_fingerprint': '', 'finish_reason': 'tool_calls'}, id='chat-40607fd279fa491888a9e9eac7158cb7', tool_calls=[{'name': 'youtube_search', 'args': {'query': 'greece'}, 'id': 'chatcmpl-tool-446bafd1e13f48ec9e19b6a8173df71b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 578, 'output_tokens': 21, 'total_tokens': 599})

In [ ]:
response.additional_kwargs

{'tool_calls': [{'id': 'chatcmpl-tool-da46f7a30e894001b202f4318d40857e',
   'type': 'function',
   'function': {'name': 'youtube_search',
    'arguments': '{"query": "greece"}'}}]}

As seen in the `tool_calls` output, the LLM correctly identifies the appropriate tool call and arguments. The LLM does not run the tool itself. We will do this in the next step.

### Execute the tool call and retrieve its output

In order to actually run the tool calls, we first need to create a ReAct agent using the prebuilt LangGraph `create_react_agent` helper method. This function creates a graph that serves as the bridge between the chat model and the available tools, thus enabling agentic tool calling. This graph is represented in the following diagram. 

***diagram pending...***

In [ ]:
agent_executor = create_react_agent(llm, tools)

We are now able to ask the agent questions that require tool calling. First, we can ask the model to return URLs to YouTube videos about Greece. We can use the `HumanMessage` class to pass the user query to the LLM.

In [ ]:
user_query = "What are some YouTube videos about Greece"
response = agent_executor.invoke({"messages": HumanMessage(content=user_query)})
response["messages"]

[HumanMessage(content='What are some YouTube videos about Greece', additional_kwargs={}, response_metadata={}, id='d6e37827-e49f-4597-a2e6-6b90f44e8dda'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-3d8464695a0443daabcd41ac7f2f4c1c', 'type': 'function', 'function': {'name': 'youtube_search', 'arguments': '{"query": "Greece"}'}}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 578, 'total_tokens': 600}, 'model_name': 'ibm/granite-3-8b-instruct', 'system_fingerprint': '', 'finish_reason': 'tool_calls'}, id='chat-4ebd2bf368584e388b271c358b16c6f6', tool_calls=[{'name': 'youtube_search', 'args': {'query': 'Greece'}, 'id': 'chatcmpl-tool-3d8464695a0443daabcd41ac7f2f4c1c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 578, 'output_tokens': 22, 'total_tokens': 600}),
 ToolMessage(content="['https://www.youtube.com/watch?v=NMlBB2pK5qo&pp=ygUGR3JlZWNl', 'https://www.youtube.com/watch?v=waQY2Ucpbd0&pp=ygUGR3JlZWNl']", name

Great! As seen in the `AIMessage`, the model correctly identified the appropriate tool call. In the `ToolMessage`, we see the model successfully returned the expected output by using the built-in LangChain YouTube tool. Finally, the `AIMessage` shows the LLM synthesized the tool response. 

Next, let's ask the model about the weather in Greece to determine whether it calls the `weather_search` tool as expected.

In [ ]:
user_query = "What is the weather in Athens, GR"
response = agent_executor.invoke({"messages": HumanMessage(content=user_query)})
response["messages"]

[HumanMessage(content='What is the weather in Athens, GR', additional_kwargs={}, response_metadata={}, id='a0c4b69c-988a-4f7d-9b8a-4780305f8e2a'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-9a0c07a3b35f4c69a351c5540ab663f8', 'type': 'function', 'function': {'name': 'weather_search', 'arguments': '{"__arg1": "Athens, GR"}'}}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 579, 'total_tokens': 605}, 'model_name': 'ibm/granite-3-8b-instruct', 'system_fingerprint': '', 'finish_reason': 'tool_calls'}, id='chat-eeed087050e049f0ad715f3615c7fdda', tool_calls=[{'name': 'weather_search', 'args': {'__arg1': 'Athens, GR'}, 'id': 'chatcmpl-tool-9a0c07a3b35f4c69a351c5540ab663f8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 579, 'output_tokens': 26, 'total_tokens': 605}),
 ToolMessage(content='In Athens, GR, the current weather is as follows:\nDetailed status: few clouds\nWind speed: 4.47 m/s, direction: 23°\nHumidity: 58%

The model was able to discern the appropriate tool to call, run the tool with the extracted arguments as well as synthesize the tool output. Now, let's ask the LLM for some suitcases under $150 for your upcoming trip. 

In [ ]:
user_query = "Find some suitcases for less than $150"
response = agent_executor.invoke({"messages": HumanMessage(content=user_query)})
response["messages"]

content='[{"products": [{"links": [{"text": "Details", "type": "pdp", "url": "https://go.ionic.click/JkGImN"}], "merchant_name": "Adorama", "merchant_product_id": "product_9102_SLAHLKY", "name": "Slinger - Slinger Leather Case for AirTag Finder Compatible with AirTags 2021 (Yellow)", "price": "$9.95", "status": "available", "thumbnail": "https://www.adorama.com/images/xlarge/SLAHLKY.JPG", "brand_name": "Slinger", "upc": "192223165557"}, {"links": [{"text": "Details", "type": "pdp", "url": "https://www.amazon.com/dp/B0B7BP6CJN?tag=ioniccommer00-20&linkCode=osi&th=1&psc=1"}], "merchant_name": "Amazon", "merchant_product_id": "B0B7BP6CJN", "name": "Veken 8 Set Packing Cubes for Suitcases, Luggage Organizer Bags Set for Carry on, Travel Gifts for Mom, Travel Essentials for Women, Travel Accessories in 4 Sizes(Extra Large, Large, Medium, Small)", "price": "$19.99", "status": "available", "thumbnail": "https://m.media-amazon.com/images/I/41ROGQM8UIL._SL160_.jpg", "brand_name": "Veken", "upc"

[HumanMessage(content='Find some suitcases for less than $150', additional_kwargs={}, response_metadata={}, id='4bc2efe6-a402-47cc-9e45-692bfcca738f'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-f92af707fa95437f846c8cd71eeedb05', 'type': 'function', 'function': {'name': 'ionic_commerce_shopping_tool', 'arguments': '{"__arg1": "suitcases, 4, 150, 1500"}'}}]}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 580, 'total_tokens': 621}, 'model_name': 'ibm/granite-3-8b-instruct', 'system_fingerprint': '', 'finish_reason': 'tool_calls'}, id='chat-184ce038c7b744128ff9a8ed7a701a9e', tool_calls=[{'name': 'ionic_commerce_shopping_tool', 'args': {'__arg1': 'suitcases, 4, 150, 1500'}, 'id': 'chatcmpl-tool-f92af707fa95437f846c8cd71eeedb05', 'type': 'tool_call'}], usage_metadata={'input_tokens': 580, 'output_tokens': 41, 'total_tokens': 621}),
 ToolMessage(content='[{"products": [{"links": [{"text": "Details", "type": "pdp", "url": "htt

As seen by the LLM response, the model correctly used the shopping tool to return several suitcases for purchase online under $150. 

# Summary

In this tutorial, you used pre-built LangChain tools to create a ReAct agent in Python with watsonx using the `granite-3-8b-instruct` model. You used the `youtube_search`, `weather_search` and `ionic_search` tools. The tutorial showed how to implement traditional tool calling as well as an agentic approach that runs the tools. 

The sample output is important as it shows the steps the agent took in creating its own agent workflow using the functions available. The tools granted to the agent were vital for answering user queries.